<h1> Question 3 - ACC Ranking </h1>



## Data Manipulation
I created two excel files:
- df has away and home team along with their scores.
- teams_df has data of teams that are competing

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.preprocessing import StandardScaler
df = pd.read_csv("/Users/apple/Downloads/ACC1819.csv", encoding = "ISO-8859-1")
df.head()

,ï»¿Date,AwayTeam,AwayScore,HomeTeam,HomeScore
0,1/1/19 13:00,Notre Dame Fighting Irish,66,Virginia Tech Hokies,81
1,1/3/19 19:00,North Carolina State Wolfpack,87,Miami (FL) Hurricanes,82
2,1/5/19 3:27,Clemson Tigers,68,Duke Blue Devils,87
3,1/5/19 12:00,Boston College Eagles,66,Virginia Tech Hokies,77
4,1/5/19 12:00,Syracuse Orange,72,Notre Dame Fighting Irish,62


In [2]:
teams_df = pd.read_csv('/Users/apple/Downloads/teams.csv')
teams_df.head()

,teams
0,Notre Dame Fighting Irish
1,North Carolina State Wolfpack
2,Clemson Tigers
3,Boston College Eagles
4,Syracuse Orange


# Methods
- Using three methods to rank teams.

## Method 1: Page Rank
- I wanted to see how much does a win matter
- So I assesed wins and losses of a team by their opponents wins and losses and their opponents opponents wins and losses ect. 
- Say team A bets team B, If B won many matched throughout, it means A had a good win and if B lost many games, it not so great win, hence a less weightage would be give.

### Implemenation: 
- I implemented this principle using Pagerank Algorithm. 
- I am taking teams as nodes and edges as each teams wins 


In [3]:
# converting teams to a list 
teams = teams_df['teams'].tolist()
teams = [team.strip() for team in teams]
n_teams = len(teams)
# Creating a new dataframe in a matrix format to store the edges
records = pd.DataFrame(np.zeros((n_teams,n_teams)),
                       columns=teams,
                       index=teams)

In [4]:
# adding wins to the dataframe
for index, game in df.iterrows():
    hometeam = game['HomeTeam']
    visitor = game['AwayTeam']
    if hometeam == visitor:
        continue

    if int(game['HomeScore']) > int(game['AwayScore']):
        # Home Team won
        records.at[hometeam, visitor] += 1
    else:
        # Visiting Team won
        records.at[visitor, hometeam] += 1

In [5]:
# Converting the dataframe to a graph 
g = records.values
G = nx.DiGraph(g)

# Using networkx package to calculate scores using page rank
pr = nx.pagerank(G, alpha=0.9)


In [6]:
# Storing scores in a dataframe
output = pd.DataFrame(list(pr.items()), index = records.index, columns=[0,'Pagerank_Score'])
output = output.drop(columns=[0])
pagerank = output.sort_values(by=['Pagerank_Score'])
pagerank

,Pagerank_Score
Virginia Cavaliers,0.011705
Duke Blue Devils,0.013203
North Carolina Tar Heels,0.013567
Virginia Tech Hokies,0.026284
Syracuse Orange,0.040092
Clemson Tigers,0.047977
North Carolina State Wolfpack,0.059077
Florida State Seminoles,0.060667
Louisville Cardinals,0.068226
Miami (FL) Hurricanes,0.083313


## Method 2: win/lose margin
- I wanted to see by how much did the team win or loss the match.
- That is winning a match with 80-65, would matter much than 66-65
- Similarly, lossing a matching with 54-55 would weigh in more than 54-70

### Implementation
- Calcluated margin by using this formula - homescore/(homescore + awayscore)

In [7]:
# calculating margin for home and away teams
df['Away_margin'] = df.AwayScore/(df.AwayScore + df.HomeScore) 
df['Home_margin'] = df.HomeScore/(df.AwayScore + df.HomeScore) 
df.head()

,ï»¿Date,AwayTeam,AwayScore,HomeTeam,HomeScore,Away_margin,Home_margin
0,1/1/19 13:00,Notre Dame Fighting Irish,66,Virginia Tech Hokies,81,0.448980,0.551020
1,1/3/19 19:00,North Carolina State Wolfpack,87,Miami (FL) Hurricanes,82,0.514793,0.485207
2,1/5/19 3:27,Clemson Tigers,68,Duke Blue Devils,87,0.438710,0.561290
3,1/5/19 12:00,Boston College Eagles,66,Virginia Tech Hokies,77,0.461538,0.538462
4,1/5/19 12:00,Syracuse Orange,72,Notre Dame Fighting Irish,62,0.537313,0.462687


## Method 3: win rate
- Win rate would determine how the team performed throughout the season.

### Implementation 
- It was pretty simple calculation - total wins/ total games played

In [8]:
# Looking for win and losses for away team and home team
df['Results'] = df.apply(lambda x: int(x['HomeScore'] > x['AwayScore']), axis=1)
df['Home_W/L'] = np.where(df['Results'] == 1, 1,0)
df['Away_W/L'] = np.where(df['Results'] == 0, 1,0)
df.head()

,ï»¿Date,AwayTeam,AwayScore,HomeTeam,HomeScore,Away_margin,Home_margin,Results,Home_W/L,Away_W/L
0,1/1/19 13:00,Notre Dame Fighting Irish,66,Virginia Tech Hokies,81,0.448980,0.551020,1,1,0
1,1/3/19 19:00,North Carolina State Wolfpack,87,Miami (FL) Hurricanes,82,0.514793,0.485207,0,0,1
2,1/5/19 3:27,Clemson Tigers,68,Duke Blue Devils,87,0.438710,0.561290,1,1,0
3,1/5/19 12:00,Boston College Eagles,66,Virginia Tech Hokies,77,0.461538,0.538462,1,1,0
4,1/5/19 12:00,Syracuse Orange,72,Notre Dame Fighting Irish,62,0.537313,0.462687,0,0,1


In [9]:
# splitting the data frame into sub data frames for away and home
away = df[[col for col in df.columns if "Away" in col]]
away.columns = ['Team', 'Score', 'Margin', 'Wins']

home = df[[col for col in df.columns if "Home" in col]]
home.columns = ['Team', 'Score', 'Margin', 'Wins']
home.head()

,Team,Score,Margin,Wins
0,Virginia Tech Hokies,81,0.551020,1
1,Miami (FL) Hurricanes,82,0.485207,0
2,Duke Blue Devils,87,0.561290,1
3,Virginia Tech Hokies,77,0.538462,1
4,Notre Dame Fighting Irish,62,0.462687,0


In [10]:
# mergeing the away and home by rowa
data = pd.concat([away,home], axis=0)

In [11]:
# Calculating total matches played by each team 
data['Total_played'] = data.groupby('Team')['Team'].transform('count')
data.head()

,Team,Score,Margin,Wins,Total_played
0,Notre Dame Fighting Irish,66,0.448980,0,20
1,North Carolina State Wolfpack,87,0.514793,1,20
2,Clemson Tigers,68,0.438710,0,19
3,Boston College Eagles,66,0.461538,0,19
4,Syracuse Orange,72,0.537313,1,20


In [12]:
# Grouping by teams 
new_df = data.groupby(['Team']).agg({'Margin': np.sum, 'Wins': np.sum, 'Total_played': np.mean})

In [13]:
new_df.head()

,Margin,Wins,Total_played
Team,,,
Boston College Eagles,9.009067,5,19
Clemson Tigers,9.727861,9,19
Duke Blue Devils,11.123952,17,21
Florida State Seminoles,10.863072,15,21
Georgia Tech Yellow Jackets,8.830893,6,19


In [14]:
# Calculating win and margin rate
new_df['win_rate'] = (new_df['Wins']/new_df['Total_played'])
new_df['margin_rate'] = (new_df['Margin']/new_df['Total_played'])

In [15]:
new_df = new_df.drop(["Margin","Wins","Total_played"], axis = 1)

In [16]:
# joning page rank scores along with win and margin rate
new_df = new_df.join(pagerank)
new_df.head()

,win_rate,margin_rate,Pagerank_Score
Team,,,
Boston College Eagles,0.263158,0.474161,0.133854
Clemson Tigers,0.473684,0.511993,0.047977
Duke Blue Devils,0.809524,0.529712,0.013203
Florida State Seminoles,0.714286,0.517289,0.060667
Georgia Tech Yellow Jackets,0.315789,0.464784,0.100768


In [17]:
# Scaling the features to bring uniformity 
scaled_features = StandardScaler().fit_transform(new_df.values)

In [18]:
scaled = pd.DataFrame(scaled_features, index=new_df.index, columns=new_df.columns)
scaled.head()

,win_rate,margin_rate,Pagerank_Score
Team,,,
Boston College Eagles,-1.002067,-0.897175,1.669027
Clemson Tigers,-0.095252,0.441348,-0.464279
Duke Blue Devils,1.351335,1.068283,-1.328101
Florida State Seminoles,0.941109,0.628744,-0.149041
Georgia Tech Yellow Jackets,-0.775363,-1.228967,0.847130


In [19]:
# Averaging all the three scores
scaled['Rating'] = (scaled['win_rate'] + scaled['margin_rate'] + scaled['Pagerank_Score'])/3

In [20]:
final_df = scaled.drop(['win_rate','margin_rate','Pagerank_Score'], axis = 1)

In [21]:
final_df.head()

,Rating
Team,
Boston College Eagles,-0.076738
Clemson Tigers,-0.039394
Duke Blue Devils,0.363839
Florida State Seminoles,0.473604
Georgia Tech Yellow Jackets,-0.385734


In [22]:
# sorting scores
rank = final_df.sort_values(by=['Rating'],ascending=False)
print(rank)

                                 Rating
Team                                   
Virginia Cavaliers             0.670699
Florida State Seminoles        0.473604
North Carolina Tar Heels       0.467723
Duke Blue Devils               0.363839
Louisville Cardinals           0.294185
Virginia Tech Hokies           0.114034
Clemson Tigers                -0.039394
Boston College Eagles         -0.076738
North Carolina State Wolfpack -0.106326
Syracuse Orange               -0.115002
Notre Dame Fighting Irish     -0.311285
Pittsburgh Panthers           -0.327297
Miami (FL) Hurricanes         -0.338245
Georgia Tech Yellow Jackets   -0.385734
Wake Forest Demon Deacons     -0.684065


In [23]:
rank["Rank"] = rank["Rating"].rank(ascending=False) 

In [24]:
rank = rank.reset_index()
rank

,Team,Rating,Rank
0,Virginia Cavaliers,0.670699,1.0
1,Florida State Seminoles,0.473604,2.0
2,North Carolina Tar Heels,0.467723,3.0
3,Duke Blue Devils,0.363839,4.0
4,Louisville Cardinals,0.294185,5.0
5,Virginia Tech Hokies,0.114034,6.0
6,Clemson Tigers,-0.039394,7.0
7,Boston College Eagles,-0.076738,8.0
8,North Carolina State Wolfpack,-0.106326,9.0
9,Syracuse Orange,-0.115002,10.0


In [25]:
#writing to csv 
rank.to_csv('/Users/apple/Desktop/ACCRankings1819.csv',index=False)

## Conclusion 
- I considered the teams over all performance in the season by calculating win rate
- I factored in a teams performance in a particular match by taking margin rate
- Even compared a teams perfromance throughout the season over other teams, by considering other teams performance in the season 